<a href="https://colab.research.google.com/github/MOOwuttichai/Cancer_Social_Listening/blob/main/%E0%B9%80%E0%B9%80%E0%B8%A2%E0%B8%81%E0%B8%A1%E0%B8%B0%E0%B9%80%E0%B8%A3%E0%B9%87%E0%B8%87%2B%E0%B9%80%E0%B8%9E%E0%B8%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install attacut
!pip install pythainlp
!pip install epitran

In [ ]:
!wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf') # Ensuring matplotlib recognizes the font
mpl.rc('font', family='TH Sarabun New')

In [ ]:
import pandas as pd
from google.colab import drive
from attacut import tokenize, Tokenizer
import pythainlp
from pythainlp import sent_tokenize, word_tokenize, Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from collections import OrderedDict
import matplotlib as mpl
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/ดึงข้อความFacbook - Sheet1.csv')

In [ ]:
data.describe()

In [ ]:
data.head(5)

In [ ]:
data.iloc[0,1]

In [ ]:
data[data['ชื่อ']==data.iloc[1,1]]

In [ ]:
comment=data.groupby('ชื่อ').sum().reset_index()

In [ ]:
comment[comment['ชื่อ']=='Wiraon Chuenchom']

In [ ]:
text = comment['คำพูดโรค'][1]

In [ ]:
text

In [ ]:
words = set(thai_words())
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
  words.add(i)
custom_tokenizer = Tokenizer(words)
Token = custom_tokenizer.word_tokenize(normalize(text))

In [ ]:
Token

In [ ]:
list_cancer = []
for i in range(len(Token)):
  if (Token[i] == "มะเร็ง")|(Token[i] == "โรคมะเร็ง"):
    list_cancer.append(Token[i]+Token[i+1])
  else :
    list_cancer.append('ไม่สามารถระบุได้')
unique_list = list(OrderedDict.fromkeys(list_cancer)) # ลบคำซ้ำ

In [ ]:
unique_list

In [ ]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปากมดลูก','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งเม็ดเลือดขาว','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งท่อน้ำดี']

In [ ]:
list_define_cancer = []
for i in range(len(unique_list)):
  for j in range(len(name_cancar)):
    if unique_list[i]==name_cancar[j]:
      list_define_cancer.append(unique_list[i])
if list_define_cancer==[]:
  list_define_cancer.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')

In [ ]:
list_define_cancer

#ลองทั้งหมด

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/ดึงข้อความFacbook - Sheet1.csv')

In [ ]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปากมดลูก','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งเม็ดเลือดขาว','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งลำไส้'
,'มะเร็งท่อน้ำดี']

In [ ]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ','ปู่ทวด','ตาทวด','ลุง','อา','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย','พ่อตา']
          ,'หญิง':['แม่','เเม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า่','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้']}

In [ ]:
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','โรค','มะเร็ง']
for i in name:
  words.add(i)

In [63]:
new_colcan = []
new_colgenden = []
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  #หาโรค
  #--------------------------------------------------------
  list_cancer = []
  for i in range(len(Token)):
    if (Token[i] == "มะเร็ง")|(Token[i] == "โรคมะเร็ง"):
      list_cancer.append(Token[i]+Token[i+1])
    # else :
    #   list_cancer.append('ไม่สามารถระบุได้')
  unique_list = list(OrderedDict.fromkeys(list_cancer))
  #----------------------------------------------------------
  list_define_cancer = []
  new_list=[]
  for i in range(len(unique_list)):
    for j in range(len(name_cancar)):
      if unique_list[i]==name_cancar[j]:
        list_define_cancer.append(unique_list[i])
  if list_define_cancer==[]:
    list_define_cancer.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  new_colcan.append(list_define_cancer)
  # for i in range(len(new_colcan)):
  #   if len(new_colcan) == 1:
  #     new_list.append(new_colcan[i][0])
  #   elif len(list_define_cancer) > 1:
  #     new_list.append('เล่ามากกว่า 2 โรค')
  # new_colcan.append(new_list)

In [64]:
new_cancer=[]
for i in range(len(new_colcan)):
  if len(new_colcan[i]) == 1:
    new_list.append(new_colcan[i][0])
  elif len(new_colcan[i]) > 1:
    new_list.append('เล่ามากกว่า 2 โรค')
  new_cancer.append(new_list)

In [ ]:
new_cancer

In [ ]:
len(new_colcan[0])

In [ ]:
use_data = comment[['ชื่อ','คำพูดโรค']]
use_data['ประเภทของมะเร็ง'] = new_cancer
use_data

#วิเคราะห์เเละสรุปผล

In [ ]:
use_data.to_csv('งาน.csv')

In [ ]:
new_list=[]
for i in range(len(use_data)):
  if len(use_data['ประเภทของมะเร็ง'][i]) == 1:
    new_list.append(use_data['ประเภทของมะเร็ง'][i][0])
  elif len(use_data['ประเภทของมะเร็ง'][i]) > 1:
    new_list.append('เล่ามากกว่า 2 โรค')
use_data['ประเภทของมะเร็ง']=new_list

In [ ]:
use_data

In [ ]:
cancer_counts = use_data['ประเภทของมะเร็ง'].value_counts()
cancer_counts

In [ ]:
set(use_data['ประเภทของมะเร็ง'])

In [ ]:
# # List of cancer names
# name_cancer = list(set(use_data['ประเภทของมะเร็ง']))
# # Count of cancer occurrences (dummy data for demonstration)
# cancer_counts = use_data['ประเภทของมะเร็ง'].value_counts()
# # Create bar plot
# plt.figure(figsize=(10, 8))
# plt.bar(name_cancer, cancer_counts.values, color='skyblue')  # Providing counts as y-axis data
# plt.xlabel('Cancer Types')
# plt.ylabel('Number of Cases')
# plt.title('Cancer Cases by Type')
# plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
# plt.tight_layout()
# plt.show()

In [ ]:
use_data['ประเภทของมะเร็ง'].value_counts().plot(kind='bar')

#หาโรคเดี่ยว

In [ ]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ','ปู่ทวด','ตาทวด','ลุง','อา','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย','พ่อตา']
          ,'หญิง':['แม่','เเม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า่','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้']}

In [ ]:
genden_list =[]
list_genden = []
for i in range(len(Token)):
  for j in range(len(Genden['หญิง'])):
    if (Token[i] == Genden['ชาย'][j]):
      list_genden.append('เพศชาย')
    elif(Token[i] == Genden['หญิง'][j]):
      list_genden.append('เพศหญิง')
genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ

In [ ]:
genden_list

In [ ]:
list_define_genden = []
for i in range(len(genden_list)):
  if len(genden_list) == 2:
    list_define_genden.append('เล่าทั้งสองเพศ')
  elif list_define_cancer==[]:
    list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  else:
    list_define_genden.append(genden_list[0])

In [ ]:
list_define_genden

#หาโรครวม

In [ ]:
 #หาเพศ
  #----------------------------------------------------------
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  list_genden = []
  for i in range(len(Token)):
    for j in range(len(Genden['หญิง'])):
      if (Token[i] == Genden['ชาย'][j]):
        list_genden.append('เพศชาย')
      elif(Token[i] == Genden['หญิง'][j]):
        list_genden.append('เพศหญิง')
  genden_list =[]
  genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ
  #-------------------------------------------------------------------
  list_define_genden = []
  for i in range(len(genden_list)):
    if len(genden_list) == 2:
      list_define_genden.append('เล่าทั้งสองเพศ')
    elif genden_list==[]:
      list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
    else:
      list_define_genden.append(genden_list[0])
    genden_list_de =[]
    genden_list_de = list(OrderedDict.fromkeys(list_define_genden))
    new_colgenden.append(genden_list_de)

In [ ]:
for i in range(len(comment)):
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(text))
  Token.append('end')
  list_genden = []
  for i in range(len(Token)):
    for j in range(len(Genden['หญิง'])):
      if (Token[i] == Genden['ชาย'][j]):
        list_genden.append('เพศชาย')
      elif(Token[i] == Genden['หญิง'][j]):
        list_genden.append('เพศหญิง')
  genden_list =[]
  genden_list = list(OrderedDict.fromkeys(list_genden))

In [ ]:
new_colgenden

In [ ]:
genden=[]
for i in range(len(new_colgenden)):
  x= set(new_colgenden[i])
  genden.append(x)

In [ ]:
len(genden)